In [1]:
from utils import *

## Extract common Zi for bulk update

In [17]:
sql_stmt = """
select * from t_zi where (layer like 'HSK_1%' or layer like 'HSK_2%') order by layer,zi;
"""

with DBConn() as _conn:
    df_hsk = pd.read_sql(sql_stmt, _conn).fillna("")

In [18]:
df_hsk.to_csv("hsk_4k.csv", index=False)

In [19]:
df_hsk

,zi,u_id,unicode,pinyin,nstrokes,alias,traditional,as_part,is_radical,layer,desc_cn,zi_en,desc_en,ts,sort_val,is_active
0,一,11789,,yi1,,,,Y,Y,HSK_1-Common-01,,,"""one/single/a (article)/as soon as/entire; who...",,161.0,Y
1,丁,3094,,ding1,,,,,,HSK_1-Common-01,,,"""male adult/the 4th of the 10 Heavenly Stems 天...",,23.0,Y
2,七,8229,,qi1,,,,,,HSK_1-Common-01,,,seven/7,,99.0,Y
3,万,10384,,wan4,,,萬,,,HSK_1-Common-01,,,ten thousand/a great number,,136.0,Y
4,丈,12770,,zhang4,,,,,,HSK_1-Common-01,,,"measure of length, ten Chinese feet (3.3 m)/to...",,181.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3875,髓,9733,,sui3,,,,,,HSK_2-CommonLow-7,,,marrow/essence/quintessence/pith (soft interio...,,3869.0,Y
3876,鬓,1814,,bin4,,,鬢,,,HSK_2-CommonLow-7,,,temples/hair on the temples,,3839.0,Y
3877,鳍,8308,,qi2,,,鰭,,,HSK_2-CommonLow-7,,,fin,,3865.0,Y
3878,鳖,1790,,bie1,,,鱉,,,HSK_2-CommonLow-7,,,freshwater soft-shelled turtle to lack/not up...,,3838.0,Y


## Compare common Zi between t_zi and t_zi_part tables

In [2]:
sql_stmt = """
select zi from t_zi where (layer like 'HSK_1%' or layer like 'HSK_2%') order by zi;
"""

with DBConn() as _conn:
    df_1 = pd.read_sql(sql_stmt, _conn)

In [3]:
df_1

,zi
0,一
1,丁
2,七
3,万
4,丈
...,...
3875,齿
3876,龄
3877,龙
3878,龟


In [4]:
sql_stmt = """
select zi from t_zi_part where is_active = 'Y' order by zi;
"""

with DBConn() as _conn:
    df_2 = pd.read_sql(sql_stmt, _conn)
df_2

,zi
0,㒸
1,㕯
2,㛂
3,㤅
4,㥏
...,...
3631,齵
3632,齾
3633,龢
3634,龤


In [5]:
sql_stmt = """
with new_zi as (
    select zi from t_zi where (layer like 'HSK_1%' or layer like 'HSK_2%')
    except 
    select zi from t_zi_part where is_active = 'Y'
)
select * from new_zi order by zi;
"""

with DBConn() as _conn:
    df_new = pd.read_sql(sql_stmt, _conn)
df_new

,zi
0,一
1,丁
2,七
3,万
4,丈
...,...
2250,齐
2251,齿
2252,龄
2253,龙


### insert into t_zi_part

In [7]:
sql_stmt = """
with new_zi as (
    select zi from t_zi where (layer like 'HSK_1%' or layer like 'HSK_2%')
    except 
    select zi from t_zi_part where is_active = 'Y'
)
insert into t_zi_part (
    zi, u_id, desc_en, is_active, hsk_note
)
select 
    t.zi, t.u_id, t.desc_en, t.is_active, 'from HSK'
from new_zi n 
join t_zi t 
on t.zi = n.zi;
"""

with DBConn() as _conn:
    db_run_sql(sql_stmt, _conn)

[DEBUG] 
with new_zi as (
    select zi from t_zi where (layer like 'HSK_1%' or layer like 'HSK_2%')
    except 
    select zi from t_zi_part where is_active = 'Y'
)
insert into t_zi_part (
    zi, u_id, desc_en, is_active, hsk_note
)
select 
    t.zi, t.u_id, t.desc_en, t.is_active, 'from HSK'
from new_zi n 
join t_zi t 
on t.zi = n.zi;



In [8]:
sql_stmt = """
with new_zi as (
    select zi from t_zi where (layer like 'HSK_1%' or layer like 'HSK_2%')
    except 
    select zi from t_zi_part where is_active = 'Y'
)
select * from new_zi order by zi;
"""

with DBConn() as _conn:
    df_test = pd.read_sql(sql_stmt, _conn)
df_test

,zi


In [6]:
sql_stmt = """
with new_zi as (
    select zi from t_zi_part where is_active = 'Y'
    except 
    select zi from t_zi where (layer like 'HSK_1%' or layer like 'HSK_2%')
)
select * from new_zi order by zi;
"""

with DBConn() as _conn:
    df_shuowen = pd.read_sql(sql_stmt, _conn)
df_shuowen

,zi
0,㒸
1,㕯
2,㛂
3,㤅
4,㥏
...,...
2394,齵
2395,齾
2396,龢
2397,龤


## Identify dup in t_zi_part table 

In [4]:
sql_stmt = """
with dup as (
	select u_id,count(*) from t_zi_part 
	where u_id is not NULL 
	group by u_id having count(*) > 1
	order by cast(u_id as int)
) 
select p.* from t_zi_part p join dup on dup.u_id = p.u_id 
order by p.u_id;
"""

with DBConn() as _conn:
    df = pd.read_sql(sql_stmt, _conn)

In [5]:
df

,zi,u_id,zi_left_up,zi_left,zi_left_down,zi_up,zi_mid,zi_down,zi_right_up,zi_right,zi_right_down,zi_mid_out,zi_mid_in,ts,desc_cn,is_active,id_shuowen
0,朓,10081,None,肉,None,None,None,None,None,兆,None,None,None,None,祭也。从肉兆聲。,Y,2658
1,朓,10081,None,月,None,None,None,None,None,兆,None,None,None,None,晦而月見西方謂之朓。从月兆聲。,Y,4296
2,鳽,11560,None,隹,None,None,None,None,None,幵,None,None,None,None,石鳥。一名雝𪆫。一曰精𠛱。从隹幵聲。《春秋傳》：“秦有士鳽 。”,Y,2290
3,鳽,11560,None,鳥,None,None,None,None,None,幵,None,None,None,None,𪁉𪂴也。从鳥幵聲。,Y,2439
4,雁,11628,None,隹,None,None,None,None,None,人,None,None,None,None,鳥也。从隹从人，厂聲。讀若鴈。,Y,2293
5,雁,11628,None,人,None,None,None,None,None,厂,None,None,None,None,鳥也。从隹从人，厂聲。讀若鴈。,Y,2293
6,薻,12596,None,艸,None,None,None,None,None,水,None,None,None,None,水艸也。从艸从水，巢聲。《詩》曰：“于以采薻？”,Y,650
7,薻,12596,None,水,None,None,None,None,None,巢,None,None,None,None,水艸也。从艸从水，巢聲。《詩》曰：“于以采薻？”,Y,650
8,胄,13147,None,肉,None,None,None,None,None,由,None,None,None,None,𦙍也。从肉由聲。,Y,2636
9,胄,13147,None,冃,None,None,None,None,None,由,None,None,None,None,兜鍪也。从冃由聲。,Y,4785


In [6]:
df.to_csv("t_zi_part_dup.csv", index=False)

In [15]:
sql_stmt = """
select *
from t_zi_part --t_part -- t_zi
;
"""

with DBConn() as _conn:
    df = db_run_sql(sql_stmt, _conn)

[DEBUG] 
select *
from t_zi_part --t_part -- t_zi
;



In [16]:
df.columns 

Index(['zi', 'u_id', 'zi_left_up', 'zi_left', 'zi_left_down', 'zi_up',
       'zi_mid', 'zi_down', 'zi_right_up', 'zi_right', 'zi_right_down',
       'zi_mid_out', 'zi_mid_in', 'ts', 'desc_cn', 'is_active'],
      dtype='object')

In [17]:
print("\n, ".join(df.columns))

zi
, u_id
, zi_left_up
, zi_left
, zi_left_down
, zi_up
, zi_mid
, zi_down
, zi_right_up
, zi_right
, zi_right_down
, zi_mid_out
, zi_mid_in
, ts
, desc_cn
, is_active


In [18]:
sql_stmt = """
select *
from w_zi_part -- w_part -- w_zi 
limit 5;
"""

with DBConn() as _conn:
    df2 = db_run_sql(sql_stmt, _conn)

[DEBUG] 
select *
from w_zi_part -- w_part -- w_zi 
limit 5;



In [20]:
print("\n, ".join(df2.columns))

zi
, zi_left_up
, zi_left
, zi_left_down
, zi_up
, zi_mid
, zi_down
, zi_right_up
, zi_right
, zi_right_down
, zi_mid_out
, zi_mid_in
, id
, ts
, uid
, is_active
, shuowen_desc


In [19]:
df2

,zi,zi_left_up,zi_left,zi_left_down,zi_up,zi_mid,zi_down,zi_right_up,zi_right,zi_right_down,zi_mid_out,zi_mid_in,id,ts,uid,is_active,shuowen_desc
0,元,None,,None,一,None,兀,None,,None,None,None,2,None,None,1,始也。从一从兀。
1,吏,None,,None,一,史,None,None,,None,None,None,5,None,None,1,治人者也。从一从史，史亦聲。
2,禮,None,示,None,None,None,None,None,豊,None,None,None,12,None,None,0,履也。所以事神致福也。从示从豊，豊亦聲。
3,祰,None,示,None,None,None,None,None,告,None,None,None,38,None,None,0,告祭也。从示从告聲。
4,祏,None,示,None,None,None,None,None,石,None,None,None,39,None,None,1,宗廟主也。《周禮》有郊、宗、石室。一曰大夫以石爲主。从示从石，石亦聲。


In [2]:
sql_stmt = """
select zi,id,sort_val,layer 
from t_zi where as_part=1 order by sort_val;
"""

with DBConn() as _conn:
    df = db_run_sql(sql_stmt, _conn)

In [4]:
df

,zi,id,sort_val,layer
0,八,314,1.0,HSK_1-Common-01
1,贝,1574,2.0,HSK_1-Common-01
2,比,1633,3.0,HSK_1-Common-01
3,卜,1911,5.0,HSK_1-Common-01
4,长,2160,12.0,HSK_1-Common-01
...,...,...,...,...
202,爫,12796,11070.0,HSK_8
203,夂,12998,11191.0,HSK_8
204,豸,13063,11228.0,HSK_8
205,丶,13185,11308.0,HSK_8


In [5]:
df.to_csv("parts-v0.csv", index=False)

In [9]:
df = pd.read_csv("t_part.csv").fillna("")

In [11]:
df.columns

Index(['Radical number', 'Radical', 'Pinyin', 'Meaning', 'Strokes',
       'Traditional', 'Examples'],
      dtype='object')

In [13]:
col_map = {
    'Radical number': "id_radical", 
    'Radical':"radical", 
    'Pinyin':"pinyin", 
    'Meaning':"meaning", 
    'Strokes':"strokes",
    'Traditional':"traditional", 
    'Examples':"example"
}

In [14]:
df.rename(columns=col_map, inplace=True)

In [28]:
df

,id_radical,radical,pinyin,meaning,strokes,traditional,example,id,is_active
0,1,一,yī,one,1,,丁 万 上 下,1,1
1,2,丨,gǔn,vertical line,1,,个 中 丰 串 临,2,1
2,3,丶,zhǔ,dot,1,,丸 义 丹 为 主,3,1
3,4,丿,piě,slash,1,,乂 乃 乎,4,1
4,4,乀,piě,slash,1,,飞,5,1
...,...,...,...,...,...,...,...,...,...
253,210,齐,qí,neat,6,,挤 剂 济,254,1
254,211,齿,chǐ,tooth,8,,龄 龅,255,1
255,212,龙,lóng,dragon,5,,龚 笼 珑,256,1
256,213,龟,guī,turtle,7,,,257,1


In [18]:
df.columns

Index(['id_radical', 'radical', 'pinyin', 'meaning', 'strokes', 'traditional',
       'example', 'id'],
      dtype='object')

In [12]:
df.index

RangeIndex(start=0, stop=258, step=1)

In [16]:
df["id"] = df.index + 1

In [21]:
df["id_radical"] = df["id_radical"].map(lambda x: int(x) if x else -1)

In [22]:
df["strokes"] = df["strokes"].map(lambda x: int(x) if x else -1)

In [24]:
df["radical"] = df["radical"].str.strip()

In [25]:
df["meaning"] = df["meaning"].str.lower()

In [27]:
df["is_active"] = 1

In [35]:
t_str = """
        id int  NOT NULL,
        id_radical int,
        radical text NOT NULL,
        traditional text,
        pinyin text,
        strokes int,
        meaning text,
        example text,
        is_active int default 0
"""

In [36]:
cols = [i.strip().split()[0] for i in t_str.split("\n") if i.strip()]

In [37]:
cols

['id',
 'id_radical',
 'radical',
 'traditional',
 'pinyin',
 'strokes',
 'meaning',
 'example',
 'is_active']

In [38]:
df2 = df[cols].copy()

In [39]:
df2

,id,id_radical,radical,traditional,pinyin,strokes,meaning,example,is_active
0,1,1,一,,yī,1,one,丁 万 上 下,1
1,2,2,丨,,gǔn,1,vertical line,个 中 丰 串 临,1
2,3,3,丶,,zhǔ,1,dot,丸 义 丹 为 主,1
3,4,4,丿,,piě,1,slash,乂 乃 乎,1
4,5,4,乀,,piě,1,slash,飞,1
...,...,...,...,...,...,...,...,...,...
253,254,210,齐,,qí,6,neat,挤 剂 济,1
254,255,211,齿,,chǐ,8,tooth,龄 龅,1
255,256,212,龙,,lóng,5,dragon,龚 笼 珑,1
256,257,213,龟,,guī,7,turtle,,1


In [41]:
with DBConn() as _conn:
    df2.to_sql("t_part", _conn, if_exists='replace', index=False)

## table=t_zi

In [65]:
sql_stmt = """
select * from t_zi where as_part=1
"""

with DBConn() as _conn:
    df4 = pd.read_sql(sql_stmt, _conn)

df4.columns 

In [67]:
df4.head()

,zi,desc_cn,pinyin,nstrokes,alias,is_radical,is_zi,has_traditional,zi_en,desc_en,id,ts,uid,is_active,layer,is_picto,is_composed,as_part,sort_id,sort_val
0,石,None,dan4,None,None,1,1,NaN,None,dry measure for grain equal to ten dou 斗[dou3]...,183,None,None,1,HSK_1-Common-02,1.0,NaN,1,215,215.0
1,木,None,mu4,None,None,1,1,NaN,None,(bound form) tree/(bound form) wood/unresponsi...,189,None,None,1,HSK_1-Common-01,NaN,NaN,1,89,89.0
2,非,None,fei1,None,None,1,1,NaN,None,to not be; not/wrong,190,None,None,1,HSK_1-Common-06,NaN,NaN,1,939,939.0
3,隶,None,li4,None,隸,1,1,1.0,None,(bound form) a person in servitude; low-ranki...,216,None,None,1,HSK_1-Common-07,NaN,1.0,1,1172,1172.0
4,大,None,da4,None,None,1,1,NaN,None,big; large,224,None,None,1,HSK_1-Common-01,NaN,NaN,1,19,19.0


In [68]:
print("\n, ".join(df4.columns ))

zi
, desc_cn
, pinyin
, nstrokes
, alias
, is_radical
, is_zi
, has_traditional
, zi_en
, desc_en
, id
, ts
, uid
, is_active
, layer
, is_picto
, is_composed
, as_part
, sort_id
, sort_val


## table=t_zi_part

In [69]:
sql_stmt = """
select * from t_zi_part where is_active=1
"""

with DBConn() as _conn:
    df5 = pd.read_sql(sql_stmt, _conn)

df5.columns 

Index(['zi', 'zi_left_up', 'zi_left', 'zi_left_down', 'zi_up', 'zi_mid',
       'zi_down', 'zi_right_up', 'zi_right', 'zi_right_down', 'zi_mid_out',
       'zi_mid_in', 'id', 'ts', 'uid', 'is_active', 'shuowen_desc'],
      dtype='object')

In [70]:
print("\n, ".join(df5.columns ))

zi
, zi_left_up
, zi_left
, zi_left_down
, zi_up
, zi_mid
, zi_down
, zi_right_up
, zi_right
, zi_right_down
, zi_mid_out
, zi_mid_in
, id
, ts
, uid
, is_active
, shuowen_desc


In [75]:
print("\n OR ".join([f"{c} like '%{{search_term}}%'" for c in df5.columns if c.startswith("zi")] ))

zi like '%{search_term}%'
 OR zi_left_up like '%{search_term}%'
 OR zi_left like '%{search_term}%'
 OR zi_left_down like '%{search_term}%'
 OR zi_up like '%{search_term}%'
 OR zi_mid like '%{search_term}%'
 OR zi_down like '%{search_term}%'
 OR zi_right_up like '%{search_term}%'
 OR zi_right like '%{search_term}%'
 OR zi_right_down like '%{search_term}%'
 OR zi_mid_out like '%{search_term}%'
 OR zi_mid_in like '%{search_term}%'


In [71]:
df5.head()

,zi,zi_left_up,zi_left,zi_left_down,zi_up,zi_mid,zi_down,zi_right_up,zi_right,zi_right_down,zi_mid_out,zi_mid_in,id,ts,uid,is_active,shuowen_desc
0,元,None,,None,一,None,兀,None,,None,None,None,2,None,None,1,始也。从一从兀。
1,吏,None,,None,一,史,None,None,,None,None,None,5,None,None,1,治人者也。从一从史，史亦聲。
2,祏,None,示,None,None,None,None,None,石,None,None,None,39,None,None,1,宗廟主也。《周禮》有郊、宗、石室。一曰大夫以石爲主。从示从石，石亦聲。
3,祝,None,示,None,None,None,None,None,人,None,None,None,47,None,None,1,祭主贊詞者。从示从人口。一曰从兌省。《易》曰：“兌爲口爲巫。”。
4,祟,None,示,None,None,None,None,None,出,None,None,None,68,None,None,1,神禍也。从示从出。


## compare radicals

In [58]:
sql_stmt = """
--select zi from t_zi where as_part=1
--except
select radical as zi,traditional as zi_tr from t_part 
order by id_radical
"""

with DBConn() as _conn:
    df3 = pd.read_sql(sql_stmt, _conn)

In [60]:
df3["zi2"] = df3["zi"] + df3["zi_tr"]

In [61]:
df3

,zi,zi_tr,zi2
0,一,,一
1,丨,,丨
2,丶,,丶
3,丿,,丿
4,乀,,乀
...,...,...,...
251,齐,,齐
252,齿,,齿
253,龙,,龙
254,龟,,龟


In [76]:
parts  = df3["zi2"].to_list()
out = []
for i in range(0, len(parts), 20):
    i_st = i 
    i_sp = i + 20
    out.append(str(i).zfill(3) + " " + "".join(parts[i_st:i_sp]))

In [77]:
out

['000 一丨丶丿乀乁乙乚亅二亠人亻儿入八丷冂冖冫',
 '020 几凵刂刁刀力勹匕匚匸十卜卩㔾厂厶又口囗土',
 '040 士夂夊夕大女子宀寸小⺌⺍尢尸屮山川巛工己',
 '060 巾干幺广廴廾弋弓彐彑彡彳忄心戈户戶扌手支',
 '080 攴攵文斗斤方无旡日曰朩木欠止歹殳母比毛氏',
 '100 气氵水氺火灬爪爫父爻爿片牙牛牜⺧犭犬玄王',
 '120 玉瓜瓦甘生用田疋𤴔疒癶白皮皿目矛矢石礻示',
 '140 禸禾穴立⺮竹米纟糹 糸缶网罒罓羊⺶⺷羽耂老而',
 '160 耒耳聿月肉臣自至臼舌舛舟艮色艹虍虫血行衤',
 '180 衣覀西见見 角讠訁 言谷豆豕豸贝貝 赤走足身车車 辛辰辶',
 '200 阝邑酉釆里钅釒 金长長 门門 阜⻏隶隹雨青非面革韦韭',
 '220 音页頁 风風 飞飛 食饣飠 首香马馬 骨高髙 髟鬥鬯鬲鬼鱼魚 鸟鳥 卤鹿',
 '240 麦麻黄黍黑黹黾鼎鼓鼠鼻齐齿龙龟龠']

In [12]:
one2nine = str(list(range(1,10)))
where_clause = f""" 
    and (
        strokes is null or cast(strokes as int) not 
            in {one2nine.replace("[","(").replace("]",")")}
    )
"""
print(where_clause)

 
    and (
        strokes is null or cast(strokes as int) not 
            in (1, 2, 3, 4, 5, 6, 7, 8, 9)
    )



### convert `x`to `X`

In [13]:
cols = [
    "zi_left_up", "zi_up", "zi_right_up",
    "zi_left", "zi_mid", "zi_right",
    "zi_left_down", "zi_down", "zi_right_down",
    "zi_mid_in", "zi_mid_out",
]

In [14]:
for c in cols:
    sql = f""" update t_zi_part set {c} = 'X' where {c} = 'x'; """
    print(sql)

 update t_zi_part set zi_left_up = 'X' where zi_left_up = 'x'; 
 update t_zi_part set zi_up = 'X' where zi_up = 'x'; 
 update t_zi_part set zi_right_up = 'X' where zi_right_up = 'x'; 
 update t_zi_part set zi_left = 'X' where zi_left = 'x'; 
 update t_zi_part set zi_mid = 'X' where zi_mid = 'x'; 
 update t_zi_part set zi_right = 'X' where zi_right = 'x'; 
 update t_zi_part set zi_left_down = 'X' where zi_left_down = 'x'; 
 update t_zi_part set zi_down = 'X' where zi_down = 'x'; 
 update t_zi_part set zi_right_down = 'X' where zi_right_down = 'x'; 
 update t_zi_part set zi_mid_in = 'X' where zi_mid_in = 'x'; 
 update t_zi_part set zi_mid_out = 'X' where zi_mid_out = 'x'; 


### 汉字部件全表及组字例

In [33]:
parts = """
1、凹——兕 2、八——分只叭 3、巴——把爬
4、白——伯迫 5、半——伴叛 6、办——协胁
7、贝——坝贫 8、本——体笨 9、币——
10、 必——泌密11、丙——柄炳12、秉——
13、 卜——扑朴14、不——歪还15、才——财豺
16、 册——删17、叉——杈钗18、长——张怅
19、 厂——厌仄产20、车——阵连21、臣——藏臧
22、 辰——晨辱23、成——城诚盛24、承——
25、 尺——尽迟26、斥——诉拆27、丑——扭钮
28、 川——钏氚29、串——窜患30、垂——锤陲
31、 寸——夺时32、大——夯达33、歹——列残
34、 丹——彤35、刀——切召36、电——俺庵
37、 刁——叼38、丁——顶宁39、东——冻栋
40、 而——耍耐41、儿——允兄先42、耳——取聂
43、 乏——泛砭44、凡——矾巩45、反——返叛
46、 方——芳防47、非——菲悲48、丰——沣缝
49、 夫——替扶50、弗——佛拂51、甫——捕傅
52、 飞——53、甘——某甜54、戈——划找
55、 个——56、更——梗便57、工——贡汞
58、 弓——张粥引59、瓜——呱胍60、广——庆扩
61、 果——颗裹62、干——竿赶63、禾——和秋
64、 乎——呼烀65、户——房芦66、火——炎烦
67、 互——冱68、亥——刻该69、及——级极
70、 几——虎风叽71、甲——钾匣狎72、见——笕现
73、 斤——近折74、今——吟矜75、韭——薤瀣
76、 九——旭杂77、巨——炬拒78、己——纪起
79、 夹——侠狭80、柬——谏81、介——价界
82、 巾——帅席83、井——进阱84、久——玖灸
85、 臼——鼠舁倪86、口——嘴叫87、开——刑形
88、 亏——跨污89、来——徕莱90、乐——
91、 里——狸厘92、吏——使93、立——位泣童
94、 隶——康逮95、力——加边96、良——粮浪
97、 两——俩满98、了——辽疗99、令——冷怜
100、龙——庞袭 101、卵——孵 102、马——闯驱
103、毛——毯撬 104、矛——袤茅 105、么——麽
106、米——粪迷 107、门——们闷 108、灭——
109、民——眠泯 110、皿——温蛊 111、末——沫抹
112、母——海每 113、目——冒瞿 114、木——树杉森
115、乃——仍奶 116、内——肉呐 117、函——涵菡
118、鸟——鸡鸣 119、牛——牢犟 120、农——浓侬
121、女——好妈汝 122、乓—— 123、皮——疲波
124、片——牌 125、乒—— 126、平——萍砰
127、七——柒切 128、乞——吃迄 129、气——汽忾
130、千——钎纤 131、欠——饮欢 132、且——沮蛆
133、丘——兵岳 134、求——球救 135、曲——蛐
136、犬——伏臭 137、冉——髯苒 138、壬——任饪
139、人——从合走 140、刃——忍纫韧 141、日——明晶昕
142、入——籴汆 143、失——迭佚 144、尸——屎尾
145、肃——箫啸 146、史—— 147、士——志仕喜
148、事—— 149、氏——舐 150、手——掰拿
151、书—— 152、术——述怵 153、束——速悚
154、戍——蔑篾 155、甩—— 156、水——淼泵踏
157、巳——导异 158、丧—— 159、山——汕岁嵩
160、上——卡让 161、勺——约药芍 162、少——吵劣妙
163、申——伸神审 164、身——射躯躬 165、生——性笙
166、升—— 167、太——态呔汰 168、天——昊忝舔
169、凸—— 170、土——地封硅 171、屯——吨沌
172、瓦——瓷瓯 173、丸——执纨 174、万——迈
175、王——班皇现 176、亡——忙忘芒 177、韦——伟苇
178、为——伪妫 179、未——妹味 180、卫——
181、我——俄娥 182、无——抚芜 183、乌——呜坞钨
184、毋—— 185、五——伍吾 186、午——忤许仵
187、戊——藏茂威 188、勿——忽惕易 189、西——牺茜洒
190、夕——矽穸汐 191、习——翦翅翩 192、下——吓虾忑
193、乡——雍飨 194、小——尖尔雀 195、兴——举誉
196、心——沁悲虑 197、戌—— 198、血——恤衅洫
199、牙——邪呀芽 200、丫—— 201、亚——哑普恶
202、严——俨酽 203、央——泱盎 204、也——他池地
205、业——邺蹼璞 206、曳——拽 207、一——疆合丽
208、夷——咦姨 209、已—— 210、乙——忆亿
211、亦——恋 212、义——仪蚁 213、尹——君笋
214、永——泳咏 215、用——拥佣 216、尤——优忧
217、由——油邮 218、酉——酒遒 219、又——对戏嫂
220、于——竽 221、予——预序 222、雨——露漏霰
223、与——写屿 224、禹—— 225、玉——国宝
226、曰——冒昌 227、再—— 228、乍——炸怎笮
229、丈——仗杖 230、兆——逃桃 231、正——歪整政
232、之——芝 233、止——址齿 234、中——忠衷
235、重——董懂 236、舟——船搬 237、州——洲
238、朱——珠侏株 239、竹—— 240、爪——爬笊抓
241、专——传砖转 242、子——孟字仔 243、自——息咱
244、月——明朋胃 245、世——屉泄 246、十——叶喷
247、丌——鼻箅淠 248、兀——光元 249、丐——钙
250、廿——革勤 251、卅——带 252、丕——坯邳
253、丞——拯蒸 254、禺——偶寓隅 255、匕——比颖疑
256、乇——托姹宅 257、夭——妖乔桥 258、氐——底抵砥
259、乜—— 260、芈—— 261、佥——检签剑
262、兮—— 263、弋——代式甙 264、彳——行街很
265、闩—— 266、爿—— 267、孑——
268、孓—— 269、幺——幼幻 270、戋——线浅笺
271、聿——律肆津 272、耒——耕耙 273、艮——跟根哏
274、豸——豹豺貌 275、氕—— 276、疋——楚蛋疑
277、臾——腴庾谀 278、豕——家逐遂
279、亠——产京主 280、丬——将壮酱 281、宀——宝寝宁
282、疒——病疲疗 283、冂——同冈炯 284、屮——出拙
285、冖——冗写冠 286、囗——国围团 287、巛——巡巢邕
288、刂——刻别划 289、卩——即节叩 290、忄——忖忆忙
291、攵——政收改 292、犭——狗犹获 293、礻——礼神祈
294、纟——红级约 295、饣——饮饿蚀 296、廴——建廷延
297、廾——弄舁异 298、彐——寻雪妇 299、虍——虎虚虑
300、勹——包句笱 301、冫——冻冰冷 302、匚——框筐匝
303、彡——形彤髯 304、讠——论认该 305、厶——私鬼允
306、阝——队阵郭 307、辶——远辽迈 308、夂——务冬条
309、灬——热蒸烈 310、衤——初衬衫 311、钅——铁银铝
312、——春秦泰 313、祭头祭察嚓 314、ㄨ——杀赵区
315、龷——共巷昔 316、釆——释番釉 317、衣底衣衷装
318、牜——牺牲物 319、⺮——笑笕笼 320、罒——罚罨罴
321、龹——眷倦鲞 322、癶——登凳癸 323、龶——青表责
324、——有灰左 325、⺈——角解兔 326、丷——关单曾
327、爫——爱受奚 328、⻊——跑跳趵 329、⺌——党尚肖
330、氵——沙河湿 331、亻——伏何作 332、扌——打抨扛
333、艹——草苗蔡
"""

In [21]:
s = """
1、凹——兕 2、八——分只叭 3、巴——把爬
"""

In [31]:
def remove_noise_word(s, noise_words=[]):
    for w in noise_words:
        s = s.replace(w, "").strip()
    return s
    
def parse_parts(s, sep="——", noise_words=["\ue80b"]):
    import re
    x = [] 
    for i in s.split("、"):
        i = re.sub(r"\d", "", i)   # strip digits
        i = remove_noise_word(i, noise_words)
        if not i or sep not in i: continue
        x.append([k.strip() for k in i.split(sep) if k.strip()])
    return x

In [32]:
parse_parts(s)

[['凹', '兕'], ['八', '分只叭'], ['巴', '把爬']]

In [34]:
parse_parts(parts)

[['凹', '兕'],
 ['八', '分只叭'],
 ['巴', '把爬'],
 ['白', '伯迫'],
 ['半', '伴叛'],
 ['办', '协胁'],
 ['贝', '坝贫'],
 ['本', '体笨'],
 ['币'],
 ['必', '泌密'],
 ['丙', '柄炳'],
 ['秉'],
 ['卜', '扑朴'],
 ['不', '歪还'],
 ['才', '财豺'],
 ['册', '删'],
 ['叉', '杈钗'],
 ['长', '张怅'],
 ['厂', '厌仄产'],
 ['车', '阵连'],
 ['臣', '藏臧'],
 ['辰', '晨辱'],
 ['成', '城诚盛'],
 ['承'],
 ['尺', '尽迟'],
 ['斥', '诉拆'],
 ['丑', '扭钮'],
 ['川', '钏氚'],
 ['串', '窜患'],
 ['垂', '锤陲'],
 ['寸', '夺时'],
 ['大', '夯达'],
 ['歹', '列残'],
 ['丹', '彤'],
 ['刀', '切召'],
 ['电', '俺庵'],
 ['刁', '叼'],
 ['丁', '顶宁'],
 ['东', '冻栋'],
 ['而', '耍耐'],
 ['儿', '允兄先'],
 ['耳', '取聂'],
 ['乏', '泛砭'],
 ['凡', '矾巩'],
 ['反', '返叛'],
 ['方', '芳防'],
 ['非', '菲悲'],
 ['丰', '沣缝'],
 ['夫', '替扶'],
 ['弗', '佛拂'],
 ['甫', '捕傅'],
 ['飞'],
 ['甘', '某甜'],
 ['戈', '划找'],
 ['个'],
 ['更', '梗便'],
 ['工', '贡汞'],
 ['弓', '张粥引'],
 ['瓜', '呱胍'],
 ['广', '庆扩'],
 ['果', '颗裹'],
 ['干', '竿赶'],
 ['禾', '和秋'],
 ['乎', '呼烀'],
 ['户', '房芦'],
 ['火', '炎烦'],
 ['互', '冱'],
 ['亥', '刻该'],
 ['及', '级极'],
 ['几', '虎风叽'],
 ['甲', '钾匣狎'],
 ['见', '笕现'],
 ['斤', '近折'],
 ['今',